In [1]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [9]:
# Load dataset
dataset_path = r"D:\BE\dl\archive\PlantVillage"
categories = os.listdir(dataset_path)
num_classes = len(categories)
image_size = 128
X = []
y = []


FileNotFoundError: ignored

In [3]:
for category in categories:
  folder_path = os.path.join(dataset_path, category)
  label = categories.index(category)
  for img_name in os.listdir(folder_path):
    img_path = os.path.join(folder_path, img_name)
    image = cv2.imread(img_path)
    if image is not None: # Check if the image is not empty
      image = cv2.resize(image, (image_size, image_size))
      X.append(image)
      y.append(label)


IndentationError: ignored

In [4]:
# Convert lists to arrays
X = np.array(X)
y = to_categorical(y, num_classes)
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


NameError: ignored

In [5]:
# Create CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size, image_size, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))


NameError: ignored

In [6]:
# Compile and train the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


NameError: ignored

In [ ]:
# Save the trained model
model.save("plant_disease_model.h5")
from keras.models import load_model
# Load the saved model
model = load_model("plant_disease_model.h5")


In [ ]:
# Provide the path to your custom input image
input_image_path = r"C:\Users\Aditya\Desktop\Machinelearning\Deeplearning\test(tH).JPG"


In [ ]:
def preprocess_image(image_path):
  try:
    image = cv2.imread(image_path)
    if image is None:
      raise Exception("Failed to read the image file.")
      image = cv2.resize(image, (image_size, image_size))
      image = image / 255.0 # Normalize the pixel values between 0 and 1
      return image
  except Exception as e:
    print("Error preprocessing image:", str(e))
    return None

In [7]:
# Function to make predictions on the input image
def predict_image(image_path):
  # Preprocess the input image
  image = preprocess_image(image_path)
  if image is not None:
    image = np.expand_dims(image, axis=0) # Add an extra dimension for batch size
    # Make the prediction
    prediction = model.predict(image)
    class_index = np.argmax(prediction)
    class_label = categories[class_index]
    return class_label
  else:
    return None

IndentationError: ignored

In [8]:
# Make a prediction on the input image
predicted_class = predict_image(input_image_path)
if predicted_class is not None:
# Print the predicted class
print("Predicted Class:", predicted_class)


IndentationError: ignored